In [ ]:
import pandas as pd
from pandas import *
import numpy as np
import requests
from pprint import pprint
import csv


token =''

keywords = 'data science,data engineer,Data Analyst,mlops,Machine Learning'

headers = {'X-Api-App-Id' : token} #токен полученный на сайте

response = requests.get('https://api.superjob.ru/2.0/vacancies/?keywords=%s' % keywords, headers=headers)

response_json = response.json()['objects']
list_of_vac = []
for item in response_json:
    list_of_vac.append([
        item['firm_name'],
        item['profession'],
        item['town']['title'],
        item['place_of_work']['title'],
        str(item['payment_from'] if item['payment_from'] else 'Договорная'),
        item['type_of_work']['title'],
        item['experience']['title'],
        item['candidat'],
        item['date_published']
    ])

list_of_columns = ['company', 'position', 'location', 'format', 'salary', 'schedule', 'levels', 'techstack', 'date_publish']

df = pd.DataFrame(list_of_vac, columns=list_of_columns)
df['date_publish'] = pd.to_datetime(df['date_publish'], unit='s')
df['source_id'] = 'superjob.ru'

with open('frame.csv', newline='', encoding='UTF8') as f:
    reader=csv.reader(f)
    frame = [x for y in list(reader) for x in y]


def tech_skills(stack):
    stack_out = []

    for item in frame:
        if item in stack.lower():
            stack_out.append(item)
    return stack_out

df['techstack'] = df['techstack'].apply(lambda x: tech_skills(x))

df.to_csv('superjob_out.csv', encoding='UTF8', index=False)